In [ ]:
import os, shutil
train_dir = '../train'
validation_dir = '../validation'
test_dir = '../test'

In [ ]:
from keras.utils import image_dataset_from_directory

IMG_SIZE = 150

train_dataset = image_dataset_from_directory(train_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE))
validation_dataset = image_dataset_from_directory(validation_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE))
test_dataset = image_dataset_from_directory(test_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE))

In [ ]:
import tensorflow as tf

# Função para calcular o tamanho do dataset
def calculate_dataset_size(dataset):
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    return dataset_size

# Função para dividir o dataset em três subsets
def split_dataset(dataset, fractions):
    dataset_size = calculate_dataset_size(dataset)
    split_sizes = [int(fraction * dataset_size) for fraction in fractions]
    
    dataset_1 = dataset.take(split_sizes[0])
    dataset_2 = dataset.skip(split_sizes[0]).take(split_sizes[1])
    dataset_3 = dataset.skip(split_sizes[0] + split_sizes[1])
    
    return dataset_1, dataset_2, dataset_3

# Frações do dataset que você quer dividir
fractions = [0.4, 0.3, 0.3]

# Dividir o dataset de treino e validacao em três subsets
train_dataset_1, train_dataset_2, train_dataset_3 = split_dataset(train_dataset, fractions)
validation_dataset_1, validation_dataset_2, validation_dataset_3 = split_dataset(validation_dataset, fractions)

# Calculando os tamanhos dos subsets de treino
split_train_size_1 = calculate_dataset_size(train_dataset_1)
split_train_size_2 = calculate_dataset_size(train_dataset_2)
split_train_size_3 = calculate_dataset_size(train_dataset_3)

# Calculando os tamanhos dos subsets de validação
split_validation_size_1 = calculate_dataset_size(validation_dataset_1)
split_validation_size_2 = calculate_dataset_size(validation_dataset_2)
split_validation_size_3 = calculate_dataset_size(validation_dataset_3)

# Mostrar os tamanhos dos subsets de treino
print(f"Tamanho do subset 1 do conjunto de treino: {split_train_size_1}")
print(f"Tamanho do subset 2 do conjunto de treino: {split_train_size_2}")
print(f"Tamanho do subset 3 do conjunto de treino: {split_train_size_3}")

# Mostrar os tamanhos dos subsets de validação
print(f"Tamanho do subset 1 do conjunto de validação: {split_validation_size_1}")
print(f"Tamanho do subset 2 do conjunto de validação: {split_validation_size_2}")
print(f"Tamanho do subset 3 do conjunto de validação: {split_validation_size_3}")

In [ ]:
from keras.applications.vgg19 import VGG19

conv_base = VGG19(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)) 
conv_base.trainable = True

# Deixar todas as camadas, exceto as últimas quatro, não treináveis (congeladas)
for layer in conv_base.layers[-5:]: 
    layer.trainable = False

In [ ]:
from tensorflow import keras

#Adicionar o modelo treinado com feature extraction
base_model = keras.models.load_model('models/ModelT_transferLearning_featureExtraction_WithoutDataAumentation_OnlyClassification.h5')

base_model.summary()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

#Criar uma camada de input com o mesmo shape da vgg19
input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

#Passar a camada de input para o modelo base VGG19
vgg19_output = conv_base(input_layer)

#Passar a saída do VGG19 para do modelo salvo
model_output = base_model(vgg19_output)

#Criar o novo modelo combinado com a vgg19 e o modelo pré treinado
model= Model(inputs=input_layer, outputs=model_output)

# Mostrar os modelos combinados num só
model.summary()

In [ ]:
# Função para imprimir o estado das camadas de forma recursiva
def print_layer_trainable_status(layer, indent=0):
    print(f"{' ' * indent}Layer: {layer.name}, Trainable: {layer.trainable}")
    if isinstance(layer, Model):
        for sub_layer in layer.layers:
            print_layer_trainable_status(sub_layer, indent + 2)

print_layer_trainable_status(model)

In [ ]:
#A partir deste bloco iremos treinar o modelo para os sub datasets
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-5, weight_decay=1e-1),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

In [ ]:
#Subset 2
history = model.fit(train_dataset_2, epochs=7, validation_data=validation_dataset_2, batch_size=64, callbacks=[early_stopping, reduce_lr])

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#Subset 1
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=0.5),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

history = model.fit(train_dataset_1, epochs=2, validation_data=validation_dataset_1, batch_size=64, callbacks=[early_stopping, reduce_lr]) 

In [ ]:
#Subset 3 -> Continuar com os mesmos valores
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=0.5),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

history = model.fit(train_dataset_3, epochs=4, validation_data=validation_dataset_3, batch_size=64, callbacks=[early_stopping, reduce_lr]) 

In [ ]:
model.save('models/ModelT_transferLearning_fineTuning_WithoutDataAumentation.h5')

In [ ]:
#Vamos colocar mais 5 camadas da vgg como treinaveis, ou seja o bloco 4 para ajustar os pesos
for layer in conv_base.layers[-10:]:
    layer.trainable = True
 
# Função para imprimir para verificar se o bloco 4 ficou com Trainable=true
def print_layer_trainable_status(model):
    for layer in model.layers:
        print(f"Layer: {layer.name}, Trainable: {layer.trainable}")

print_layer_trainable_status(conv_base)

In [ ]:
# Função para imprimir o estado das camadas de forma recursiva
def print_layer_trainable_status(layer, indent=0):
    print(f"{' ' * indent}Layer: {layer.name}, Trainable: {layer.trainable}")
    if isinstance(layer, Model):
        for sub_layer in layer.layers:
            print_layer_trainable_status(sub_layer, indent + 2)

print_layer_trainable_status(model)

Continuar com os treinos para ver se o modelo tem melhorias

In [ ]:
#Subset 3 -> Continuar com os mesmos valores
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=0.5),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

history = model.fit(train_dataset_3, epochs=4, validation_data=validation_dataset_3, batch_size=64, callbacks=[early_stopping, reduce_lr]) 

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-5, weight_decay=0.5),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

history = model.fit(train_dataset_1, epochs=5, validation_data=validation_dataset_1, batch_size=64, callbacks=[early_stopping, reduce_lr]) 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(14, 6))

# Gráfico da Accuracya
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Accuracy no Treino')
plt.plot(epochs, val_acc, 'b-', label='Accuracy na Validação')
plt.title('Accuracy no Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True) 
plt.xticks(range(1, len(acc) + 1, 2))  
plt.yticks(fontsize=12) 
plt.tight_layout()

# Gráfico da Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Loss no Treino')
plt.plot(epochs, val_loss, 'b-', label='Loss na Validação')
plt.title('Loss no Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True) 
plt.xticks(range(1, len(acc) + 1, 2))
plt.yticks(fontsize=12) 
plt.tight_layout()

# Mostrar os gráficos
plt.show()


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=0.5),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

history = model.fit(train_dataset_1, epochs=2, validation_data=validation_dataset_1, batch_size=64, callbacks=[early_stopping, reduce_lr]) 

In [ ]:
model.save('models/ModelT_transferLearning_fineTuning_WithoutDataAumentation.h5')

In [ ]:
#Vamos colocar mais 5 camadas da vgg como treinaveis, ou seja o bloco 3 para ajustar os pesos
for layer in conv_base.layers[-15:]:
    layer.trainable = True
 
# Função para imprimir para verificar se o bloco 4 ficou com Trainable=true
def print_layer_trainable_status(model):
    for layer in model.layers:
        print(f"Layer: {layer.name}, Trainable: {layer.trainable}")

print_layer_trainable_status(conv_base)

In [ ]:
# Função para imprimir o estado das camadas de forma recursiva
def print_layer_trainable_status(layer, indent=0):
    print(f"{' ' * indent}Layer: {layer.name}, Trainable: {layer.trainable}")
    if isinstance(layer, Model):
        for sub_layer in layer.layers:
            print_layer_trainable_status(sub_layer, indent + 2)

print_layer_trainable_status(model)

In [ ]:
#Aumetar o weight_decay para tentar reduzir o overfitting
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=0.5),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

history = model.fit(train_dataset_1, epochs=2, validation_data=validation_dataset_1, batch_size=32, callbacks=[early_stopping, reduce_lr]) 

In [ ]:
from tensorflow import keras

loaded_model = keras.models.load_model('models/ModelT_transferLearning_fineTuning_WithoutDataAumentation.h5')

val_loss, val_acc = loaded_model.evaluate(validation_dataset)
print('val_acc:', val_acc)

In [ ]:
val_loss, val_acc = loaded_model.evaluate(test_dataset) 
print('val_acc:', val_acc)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

def extract_features(dataset, model):
    features = []
    for batch in dataset:
        images, _ = batch
        features_batch = model.predict(images)
        features.append(features_batch)
    return np.concatenate(features, axis=0)

In [ ]:
loaded_model_for_getFeatures = keras.models.load_model('models/ModelT_transferLearning_fineTuning_WithoutDataAumentation.h5')

In [ ]:
train_features = extract_features(train_dataset, loaded_model_for_getFeatures)
validation_features = extract_features(validation_dataset, loaded_model_for_getFeatures)
test_features = extract_features(test_dataset, loaded_model_for_getFeatures)

In [ ]:
import numpy as np

save_path = 'features_teste/ModelT_transferLearning_fineTuning_WithoutDataAumentation/'

np.save(save_path + 'train_features_dataset.npy', train_features)

np.save(save_path + 'val_features_dataset.npy', validation_features)

np.save(save_path + 'test_features_dataset.npy', test_features)